# Citysearch Agent deployed on Agentcore runtime
In this notebook, we set recreate simple chat bot we built with Strands in 03-agentic-metrics, to be deployable with AgentCore Runtime.  

## 1) Set up dependances

In [1]:
# Install Strands Agents
!pip install strands-agents strands-agents-tools pandas bedrock_agentcore_starter_toolkit


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


We'll also import a list of city data, to use as our gold standard set.
This is from: https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

Now we'll make a quick checking function.  Our agent will be trying to find population and area, so we'll compare how well it did against this list.

## 1) Lets rebuilt the a simple chat bot from 03-agentic-metrics to be deployed with AgentCore

In [2]:
%%writefile ./citysearch.py
from botocore.config import Config
import requests
from ddgs import DDGS
from strands import Agent, tool
from strands.models import BedrockModel
import pandas as pd
import re

#A custom config for Bedrock to only allow short connections - for our demo we expect all calls to be fast.
#here we turn off retries, and we time out after 20 seconds.
quick_config = Config(
    connect_timeout=5,
    read_timeout=20,
    retries={"max_attempts": 0}
)

from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.models import BedrockModel

from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

@tool
def web_search(topic: str) -> str:
    """Search Duck Duck go Service for a given topic.
    Return a string listing the top 5 results including the url, title, and description of each result.
    """
    try:
        results = DDGS().text(topic, region=region, max_results=5)
        return results if results else "No results found."
    except Exception as e:
        return f"Search error: {str(e)}"

#Create the chatbot.  We'll use Nova Micro to optimize for latency, cost, and capacity
chatbot_model_name = "us.amazon.nova-micro-v1:0"
#add custom timeout for the model, to keep the tool from hanging or retrying too much.
chatbot_model = BedrockModel(
    model_id=chatbot_model_name,
    boto_client_config=quick_config    
)
chatbot = Agent(tools=[web_search], model=chatbot_model)
#Call the chat bot with a simple request.
prompt = """How many people live in New York, and what's the area of the city in square miles?
After you respond, also include your answer in 'pop' and 'area' XML tags, for programatic processing.
The values in the XML tags should only be numbers, no words or commas."""
chatbot_response = chatbot(prompt)

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####

@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime entrypoint function"""
    user_input = payload.get("prompt", "")

    # Invoke the agent
    response = chatbot(user_input)
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####


Writing ./citysearch.py


# 2) Lets configure the agent with AgentCore Runtime

In [3]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "citysearch"
response = agentcore_runtime.configure(
    entrypoint="citysearch.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="05-02-03-requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/citysearch.py, bedrock_agentcore_name=citysearch
Configuring BedrockAgentCore agent: citysearch


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/Dockerfile
Generated .dockerignore: /Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/.dockerignore
Setting 'citysearch' as default agent
Bedrock AgentCore configured: /Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/Dockerfile'), dockerignore_path=PosixPath('/Users/skoppar/workspace/sample-gen-ai-evaluations-workshop/05-framework-specific-evaluations/05-02-AgentCore/.dockerignore'), runtime='Docker', region='us-east-1', account_id='146666888814', execution_role=None, ecr_repository=None, auto_create_ecr=True)

# 3) Now that the citysearch agent is configured with AgentCore, lets launch it. 
This step 
1. Publishes the Docker file as a container into ECR, 
2. Creates a new generates an version for the agent. If you run this multiple times you will see multiple versions of the Agent
3. Deploys the container on AgentCore Runtime
This creates / updates the AgentCore runtime for the agent "citysearch" to have the latest version of the agent.

In [5]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'citysearch' to account 146666888814 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 146666888814.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-citysearch
Using execution role from config: arn:aws:iam::146666888814:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-672e22fb7a
Preparing CodeBuild project and uploading source...
Using CodeBuild role from config: arn:aws:iam::146666888814:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-672e22fb7a
Using .dockerignore with 44 patterns
Uploaded 

# 4) Check the status of the deployed agent to ensure agent is ready for inference at the endpoint

In [6]:
## Check agent status
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: citysearch


'READY'

# 5) Test the agent to ensure it is ready for evaluation

In [7]:
## Test successful deployment of the agent
import boto3
import json
import uuid
import textwrap

# Create a session ID for demonstrating session continuity with AgentCore
session_id = uuid.uuid4()

client = boto3.client('bedrock-agentcore', region_name='us-east-1')
payload =  json.dumps({"prompt": "How many people live in New York, and what's the area of the city in square miles?"})

response = client.invoke_agent_runtime(
    agentRuntimeArn=launch_result.agent_arn,
    runtimeSessionId=f"eval-session-{uuid.uuid4()}",  # Must be 33+ chars
    payload=payload,
    qualifier="DEFAULT" # Optional
)
response_body = response['response'].read()
response_data = json.loads(response_body)
formatted = json.dumps(response_data, indent=2)
wrapped = textwrap.fill(formatted, width=80)
print(wrapped)



"As of July 2024, the population of New York City is approximately 8,478,000
people, according to the Department of City Planning. The total area of New York
City is approximately 468.9 square miles, encompassing five boroughs: Manhattan,
Brooklyn, Queens, The Bronx, and Staten Island.\n\nHere is the information in
'pop' and 'area' XML tags:\n\n<pop>8478000</pop>\n<area>468.9</area>"


In [8]:
citysearch_agent_arn=launch_result.agent_arn
%store citysearch_agent_arn


Stored 'citysearch_agent_arn' (str)


In [9]:
## Lets look at the output of the agent
response

{'ResponseMetadata': {'RequestId': '1709007c-09d1-4050-a89d-afdeb746ff97',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 22 Oct 2025 23:02:38 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1709007c-09d1-4050-a89d-afdeb746ff97',
   'baggage': 'Self=1-68f9627f-6116aa173afd31452c27df31,session.id=eval-session-96c00e5a-c83e-4a08-88f3-16ab499deec3',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'eval-session-96c00e5a-c83e-4a08-88f3-16ab499deec3',
   'x-amzn-trace-id': 'Root=1-68f9627f-6776d01023f5825650bfc525;Parent=f8622f389d690a86;Sampled=1;Self=1-68f9627f-6116aa173afd31452c27df31'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'eval-session-96c00e5a-c83e-4a08-88f3-16ab499deec3',
 'traceId': 'Root=1-68f9627f-6776d01023f5825650bfc525;Parent=f8622f389d690a86;Sampled=1;Self=1-68f9627f-6116aa173afd31452c27df31',
 'baggage': 'Self=1-68f9627f-6116aa173afd31452c27df31,session.id=eval-session-96c

## Here is what we have accomplished so far:
1. We upgraded an existing agent to be deployable on Agentcore runtime
2. Tested a simple invocation of the citysearch agent running on Agentcore
3. Ensured the tool is appropriately configured through Agent Observability logs. Refer to README for screenshots which you can observe on your AWS console
4. Stored the agent arn to be able to retrieve during the evaluation step

## Now lets move on to the next portion which is agent evaluations --> 05-02-02-Agent-and-tool-evals-with-xray.ipynb